In [6]:
import os
import cv2
import numpy as np
from tensorflow.keras import layers, models

def load_data(data_dir, img_size=(128, 32)):
    images = []
    for img_file in os.listdir(data_dir):
        img_path = os.path.join(data_dir, img_file)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  
        img = cv2.resize(img, img_size)  
        images.append(img)
    return np.array(images)

  
def build_crnn_model(input_shape=(32, 128, 1)):
    inputs = layers.Input(shape=input_shape)
    nn = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    nn = layers.MaxPooling2D(pool_size=(2, 2))(nn)
    nn = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2))(nn)
    nn = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2))(nn)
    nn = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2))(nn)
    nn = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2))(nn)
    nn = layers.Reshape(target_shape=(nn.shape[1], nn.shape[2] * 512))(nn)

    rnn = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(nn)
    rnn = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(rnn)    
    outputs = layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(nn)
    model = models.Model(inputs, outputs)
    return model

def data_generator(images, batch_size=32):
    while True:
        for i in range(0, len(images), batch_size):
            batch_images = images[i:i + batch_size]
            yield batch_images, batch_images  

def train_crnn(data_dir, epochs=10, batch_size=32):
    images = load_data(data_dir)
    images = images / 255.0  # Normalize
    images = np.expand_dims(images, axis=-1)
    model = build_crnn_model()
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

    model.fit(data_generator(images, batch_size=batch_size),
              steps_per_epoch=len(images) // batch_size, 
              epochs=epochs)
    
    model.save("crnn_model.h5")

if __name__ == "__main__":
    data_dir = input("Enter the path to your dataset directory :")
    epochs = int(input("Enter the number of epochs: "))
    batch_size = int(input("Enter the batch size: "))
    train_crnn(data_dir, epochs=epochs, batch_size=batch_size)


Enter the path to your dataset directory : yugyg
Enter the number of epochs:  bb


ValueError: invalid literal for int() with base 10: 'bb'